PREGUNTAR: Puedo chorearme mi archivo punto2/contabilizado/output.txt con estructura

```sql 
id_retador puntos_total cantidad
```

para simplificar el calculo de la PP?

Primer JOB es solo para formatear el DATASET COMO UN INNER JOIN 

In [50]:
from MRE import Job
inputDir = "./jugadores/"
promedioDir = "../punto2/contabilizado/"
outputDir = "./format/"

In [ ]:
def fmap(key, value, context):
    lista = value.split()
    context.write((key,'RETADO'), lista[0]) 

def fmap2(key, value, context):
    lista = value.split()
    context.write((key, 'PP')  , (int(lista[0])/int(lista[1])))

def cmpShort(key1,key2):
    if(key1[0] == key2[0]):
        return 0
    elif(key1[0] < key2[0]):
        return -1
    else:
        return 1

def cmpShuffle(key1, key2):
    if(key1[1] == key2[1]):
        return 0
    elif(key1[1] == "PP"): #PP es el primero 
        return -1
    else:
        return 1 #Caso retado

def fred(key, values, context):
    acumular_retados = set()
    pp = next(values)  #Es un iterator pero si lo resuelvo con next() me da cualquier cosa 
    for v in values:
        acumular_retados.add(v)
    context.write(key[0], (pp,acumular_retados,puntaje_heroico_inicial))

    
puntaje_heroico_inicial = 1

jobCount = Job(inputDir, outputDir, fmap, fred)
jobCount.setSortCmp(cmpShort)
jobCount.setShuffleCmp(cmpShuffle)
jobCount.setParams(puntaje_heroico_inicial)
jobCount.addInputPath(promedioDir,fmap2)

success = jobCount.waitForCompletion()

print(success)

True


Segundo job iterativo

In [ ]:
inputDir = "./format/"
outputDir = "./out/"

In [ ]:
def fmap(key, value, context):
    context.write(key, value)

def fred(key, values, context):
    c = 0
    for v in values:
        c += 1
    if c >= H:
        context.write(key, c)
H = 12

alfa = 0.1
jobCountBoth = Job(inputDir, outputDir, fmap, fred)
jobCount.setParams(alfa)
jobCountBoth.setParams(H)
success = jobCountBoth.waitForCompletion()

print(success)

FileNotFoundError: [Errno 2] No such file or directory: './distinct/'

Tercer Job saca top 10 